# Brand Level Eco-Score

- Loads cleaned REI dataset from i9
- Loads the updated materials_updated.json with eco-scores
- Maps each material to its eco_score
- Calculates each product's eco_score (average of materials used)
- Aggregates to compute brand-level eco-scores
```
| brand | avg_eco_score | # of products analyzed |
```

In [3]:
import pandas as pd
import json
rei = pd.read_csv("../data/cleaned_rei_data.csv")  

rei["item_materials"] = rei["item_materials"].str.lower()

In [4]:
rei.head(1)

,item_name,item_brand,item_price,item_description,item_id,item_materials
0,Darn Tough Bear Town Micro Crew Socks - Women's,Darn Tough,25.0,Lightly cushioned hiking socks with all the an...,#159260,54% nylon/43% merino wool/3% lycra spandex


In [6]:
with open("../data/materials_updated.json", "r") as f:
    materials_json = json.load(f)

In [7]:
print(materials_json)

{'natural_fibers': {'Merino_wool': {'name': 'Merino Wool', 'good_for': ['warmth', 'moisture management', 'odor resistance', 'soft feel'], 'alternative_materials': ['Wool', 'Alpaca Wool', 'Recycled Wool'], 'eco_score': 0.55}, 'Wool': {'name': 'Wool', 'good_for': ['insulation', 'durability', 'temperature regulation'], 'alternative_materials': ['Merino Wool', 'Alpaca Wool', 'Recycled Wool'], 'eco_score': 0.6}, 'Cotton': {'name': 'Cotton', 'good_for': ['breathability', 'comfort', 'absorbency'], 'alternative_materials': ['Organic Cotton', 'TENCEL', 'Hemp'], 'eco_score': 0.25}, 'Organic_cotton': {'name': 'Organic Cotton', 'good_for': ['breathability', 'softness', 'low environmental impact'], 'alternative_materials': ['TENCEL', 'Hemp', 'Recycled Cotton'], 'eco_score': 0.85}, 'Alpaca_wool': {'name': 'Alpaca Wool', 'good_for': ['luxury feel', 'insulation', 'hypoallergenic comfort'], 'alternative_materials': ['Merino Wool', 'Cashmere'], 'eco_score': 0.7}, 'TENCEL': {'name': 'TENCEL (Lyocell)', '

In [8]:
material_scores = {}

for category, contents, in materials_json.items():
    for key, attributes in contents.items():
        if isinstance(attributes, dict) and "eco_score" in attributes:
            material_scores[attributes["name"].lower().replace("_", " ")] = attributes["eco_score"]

In [9]:
print(material_scores)

{'merino wool': 0.55, 'wool': 0.6, 'cotton': 0.25, 'organic cotton': 0.85, 'alpaca wool': 0.7, 'tencel (lyocell)': 0.95, 'paper fiber': 0.5, 'nylon': 0.15, 'recycled nylon': 0.65, 'polyester': 0.2, 'recycled polyester': 0.7, 'spandex (elastane / lycra)': 0.1, 'polypropylene': 0.35, 'coolmax ecomade polyester': 0.7, 'repreve polyester': 0.8, 'polyurethane (pu)': 0.3, 'thermoplastic polyurethane (tpu)': 0.35, 'abs plastic': 0.1, 'gore-tex': 0.2, 'expanded polyethylene gore-tex (epe)': 0.7, 'ascentshell': 0.6, 'dryvent membrane': 0.55, 'keen.dry membrane': 0.5, 'omni-tech': 0.55, 'cosmo waterproof membrane': 0.5, 'phasic lt polyester': 0.65, 'leather': 0.2, 'suede leather': 0.2, 'nubuck leather': 0.2, 'synthetic leather': 0.45, 'rubber': 0.45, 'recycled rubber': 0.8, 'eva foam': 0.25, 'kevlar': 0.3, 'aluminum (7065)': 0.6, 'polyurethane coating (pu)': 0.45, 'durable water repellent (dwr)': 0.1, 'non-pfc durable water repellent': 0.85, 'fc0 durable water repellent': 0.85}


In [10]:
def compute_product_eco_score(materials_text):
    found_scores = []

    for material, score in material_scores.items():
        print(f"material {material}, materials text:  {materials_text}")
        if material in materials_text:
            found_scores.append(score)

    if len(found_scores) == 0:
        return None  # product has unknown materials or missing data

    return sum(found_scores) / len(found_scores)

In [11]:
rei["product_eco_score"] = rei["item_materials"].apply(compute_product_eco_score)

material merino wool, materials text:  54% nylon/43% merino wool/3% lycra spandex
material wool, materials text:  54% nylon/43% merino wool/3% lycra spandex
material cotton, materials text:  54% nylon/43% merino wool/3% lycra spandex
material organic cotton, materials text:  54% nylon/43% merino wool/3% lycra spandex
material alpaca wool, materials text:  54% nylon/43% merino wool/3% lycra spandex
material tencel (lyocell), materials text:  54% nylon/43% merino wool/3% lycra spandex
material paper fiber, materials text:  54% nylon/43% merino wool/3% lycra spandex
material nylon, materials text:  54% nylon/43% merino wool/3% lycra spandex
material recycled nylon, materials text:  54% nylon/43% merino wool/3% lycra spandex
material polyester, materials text:  54% nylon/43% merino wool/3% lycra spandex
material recycled polyester, materials text:  54% nylon/43% merino wool/3% lycra spandex
material spandex (elastane / lycra), materials text:  54% nylon/43% merino wool/3% lycra spandex
mat

In [20]:
brand_avg_price = (
    rei.groupby("item_brand")["item_price"].mean().reset_index()
)
brand_avg_price.head(10)

,item_brand,item_price
0,Aku,200.00000
1,Altra,190.00000
2,Arc'teryx,291.85375
3,Arms of Andes,95.00000
4,Asolo,183.91000
5,Ben's,5.95000
6,Black Diamond,115.84000
7,Buff,19.00000
8,Carolina Mfg.,6.00000
9,Columbia,72.36150


In [16]:
brand_eco_scores = (
    rei.groupby("item_brand")["product_eco_score"]
    .mean()
    .reset_index()
    .rename(columns={"product_eco_score": "brand_eco_score"})
)

brand_eco_scores = brand_eco_scores.sort_values(by="brand_eco_score", ascending=False)

In [19]:
brand_eco_scores.head(50)

,item_brand,brand_eco_score
3,Arms of Andes,0.650000
35,Royal Robbins,0.512500
15,Fjallraven,0.500000
40,Smartwool,0.466979
30,On,0.461111
27,Mountain Equipment,0.450000
25,Marmot,0.450000
1,Altra,0.450000
38,Saxx,0.450000
41,Snow Peak,0.450000


In [22]:
brand_profiles = pd.merge(brand_eco_scores, brand_avg_price, on='item_brand', how='inner')


In [27]:
brand_profiles.dropna()

,item_brand,brand_eco_score,item_price
0,Arms of Andes,0.650000,95.000000
1,Royal Robbins,0.512500,82.730000
2,Fjallraven,0.500000,152.500000
3,Smartwool,0.466979,22.011875
4,On,0.461111,213.333333
5,Mountain Equipment,0.450000,99.950000
6,Marmot,0.450000,48.730000
7,Altra,0.450000,190.000000
8,Saxx,0.450000,34.830000
9,Snow Peak,0.450000,79.950000


In [28]:
print("\n✅ Brand Level Eco-Scores:")
print(brand_eco_scores.head(10))
print("\n✅ Brand Level Prices:")
print(brand_avg_price.head(10))

brand_profiles.to_csv("../data/brand_profiles.csv", index=False)
print("\nSaved as brand_profiles.csv")


✅ Brand Level Eco-Scores:
            item_brand  brand_eco_score
3        Arms of Andes         0.650000
35       Royal Robbins         0.512500
15          Fjallraven         0.500000
40           Smartwool         0.466979
30                  On         0.461111
27  Mountain Equipment         0.450000
25              Marmot         0.450000
1                Altra         0.450000
38                Saxx         0.450000
41           Snow Peak         0.450000

✅ Brand Level Prices:
      item_brand  item_price
0            Aku   200.00000
1          Altra   190.00000
2      Arc'teryx   291.85375
3  Arms of Andes    95.00000
4          Asolo   183.91000
5          Ben's     5.95000
6  Black Diamond   115.84000
7           Buff    19.00000
8  Carolina Mfg.     6.00000
9       Columbia    72.36150

Saved as brand_profiles.csv
